# あなたの文章に合った「いらすとや」画像をレコメンド♪（勉強会編）

解説記事: https://qiita.com/sonoisa/items/775ac4c7871ced6ed4c3

## アルゴリズムの概要

本アプリの基本的なアイディアは次のとおりです。

1. 与えられた文や画像の説明文を、それぞれ文の分散表現（つまりはベクトル）に変換する。
1. 与えられた文と画像の説明文の意味の近さを、それぞれの文の分散表現を使って計算する（意味の近さ = 2つのベクトルのなす角の小ささ = コサイン類似度の大きさとする）。
1. コサイン類似度が大きい説明文を持つ画像トップN個を選ぶことで、与えられた文と意味が近い画像を発見できる。

模式図にすると、次のようになります。

<img src="https://camo.qiitausercontent.com/fb62a6b8a0fd447e1ff1370e83ff0b636a3f9a36/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f32363036322f33393935316266392d663630322d383565332d646239662d6235353764663164376266642e706e67" width="800">

## 準備

### パスワード設定

**※勉強会に用いるデータセットの解凍にはパスワードが必要です（一般には公開しません）。**  
**※次の変数 RESOURCES_PASSWORD に解凍用の秘密のパスワードを設定してから、以降の処理を実行してください。**

In [1]:
# データセットの解凍用パスワード
RESOURCES_PASSWORD = ""

### 依存ライブラリのインストール

インストールに5分程度かかります。気長にお待ちください。

In [2]:
%%time
!pip install mecab-python3==1.0.3 ipadic==1.0.0 pymagnitude

     |████████████████████████████████| 491kB 5.5MB/s 
     |████████████████████████████████| 13.4MB 8.2MB/s 
     |████████████████████████████████| 5.4MB 47.0MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=46fcd4438730aefabcfc7b0abed4cfd4a873204da0dbc3753ce626a9959c1e9a
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
  Created wheel for pymagnitude: filename=pymagnitude-0.1.143-cp37-cp37m-linux_x86_64.whl size=360622450 sha256=066a87b877acc81021b44328efd3bad55f9b91c6b782496d720ef9aeeffa4125
  Stored in directory: /root/.cache/pip/wheels/bb/b2/16/f88eb50800667ffe23d509c7e2157923a08d10ed6d6410124f
Successfully built ipadic pymagnitude
CPU times: user 688 ms, sys: 103 ms, total: 792 ms
Wall time: 4min 28s


### fastTextの学習済みモデルのダウンロード

In [3]:
!wget -O fasttext_model.tar "https://www.floydhub.com/api/v1/resources/SnBYkUGB9PdsbQMWbBb9jn?content=true&download=true"
!tar xvf fasttext_model.tar

--2021-03-21 12:59:17--  https://www.floydhub.com/api/v1/resources/SnBYkUGB9PdsbQMWbBb9jn?content=true&download=true
Resolving www.floydhub.com (www.floydhub.com)... 172.67.72.144, 104.26.1.30, 104.26.0.30, ...
Connecting to www.floydhub.com (www.floydhub.com)|172.67.72.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘fasttext_model.tar’

fasttext_model.tar      [            <=>     ] 931.35M  31.2MB/s    in 31s     

2021-03-21 12:59:48 (30.0 MB/s) - ‘fasttext_model.tar’ saved [976586752]

./
./jawiki.ipadic.fasttext.ws5-neg5-epoch5.magnitude


### 「いらすとや」さんの画像メタデータのダウンロード

In [5]:
!wget -O resources.tar "https://www.floydhub.com/api/v1/resources/Y6GaFAJ8uiiZPEZeXcpRRN?content=true&download=true"
!tar xvf resources.tar

--2021-03-21 12:59:56--  https://www.floydhub.com/api/v1/resources/Y6GaFAJ8uiiZPEZeXcpRRN?content=true&download=true
Resolving www.floydhub.com (www.floydhub.com)... 104.26.1.30, 172.67.72.144, 104.26.0.30, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.26.1.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘resources.tar’

resources.tar           [ <=>                ]   3.47M  --.-KB/s    in 0.1s    

2021-03-21 12:59:57 (26.0 MB/s) - ‘resources.tar’ saved [3634688]

./
./ii20210224.zip


### データの解凍

In [6]:
!unzip -P {RESOURCES_PASSWORD} ii20210224.zip

Archive:  ii20210224.zip
  inflating: irasuto_items.json      


### fastTextの学習済みモデルを読み込む

In [7]:
from pymagnitude import *

fasttext_model = Magnitude("jawiki.ipadic.fasttext.ws5-neg5-epoch5.magnitude", 
                           normalized=False, ngram_oov=True, case_insensitive=True)

動作確認として、有名なアナロジー問題を計算してみます。  
モデルの初回読み込みには時間がかかります。

男でいう王子は女でいう何か？というアナロジー問題です。  
結果は期待通り王女になっています。ちゃんと動いていますね。


In [8]:
%%time
similarities = fasttext_model.most_similar(positive=['王子', '女'], negative=['男'])
similarities

CPU times: user 51.4 s, sys: 1.21 s, total: 52.6 s
Wall time: 54.5 s


### 画像メタデータを読み込む

In [9]:
import json

with open('irasuto_items.json', 'r') as items_file:
    items = json.load(items_file)

## 前処理の定義

<img src="https://camo.qiitausercontent.com/89ed1670075b68629fff160b115db28eaaa74b41/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f32363036322f34336531353065382d663765342d363833392d636539372d3266626631336639313439392e706e67" width="800">

### 正規化処理の定義

neologdの正規化処理を少し変えたものを利用します。

- neologdの正規化処理: https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja

他にも色々と正規化の方法はありうるでしょう。

In [10]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    s = s.upper()
    return s

In [11]:
def normalize_text(text):
    return normalize_neologd(text)

### 形態素解析の定義

MeCabを用いて正規化済み文字列を形態素解析します。  
辞書は分散表現を学習したときと同じものであるIPA辞書を使用します。

In [12]:
import MeCab
import ipadic

mecab = MeCab.Tagger(ipadic.MECAB_ARGS)

In [13]:
class Morph(object):
    def __init__(self, surface, pos, base):
        self.surface = surface
        self.pos = pos
        self.base = base
    def __repr__(self):
        return str({
            "surface": self.surface,
            "pos": self.pos,
            "base": self.base
        })

def tokenize(sentence):
    sentence = normalize_text(sentence)
    mecab.parse("")
    lines = mecab.parse(sentence).split("\n")
    tokens = []
    for line in lines:
        elems = line.split("\t")
        if len(elems) < 2:
            continue
        surface = elems[0]
        if len(surface):
            feature = elems[1].split(",")
            base = surface if len(feature) < 7 or feature[6] == "*" else feature[6]
            pos = ",".join(feature[0:4])
            tokens.append(Morph(surface=surface, pos=pos, base=base))
    return tokens

試しに形態素解析してみます。surfaceは形態素、posは品詞、baseは原形です。

In [14]:
tokenize("MeCabを用いて正規化済み文字列を形態素解析します！！")

[{'surface': 'MECAB', 'pos': '名詞,一般,*,*', 'base': 'MECAB'},
 {'surface': 'を', 'pos': '助詞,格助詞,一般,*', 'base': 'を'},
 {'surface': '用い', 'pos': '動詞,自立,*,*', 'base': '用いる'},
 {'surface': 'て', 'pos': '助詞,接続助詞,*,*', 'base': 'て'},
 {'surface': '正規', 'pos': '名詞,形容動詞語幹,*,*', 'base': '正規'},
 {'surface': '化', 'pos': '名詞,接尾,サ変接続,*', 'base': '化'},
 {'surface': '済み', 'pos': '名詞,接尾,一般,*', 'base': '済み'},
 {'surface': '文字', 'pos': '名詞,一般,*,*', 'base': '文字'},
 {'surface': '列', 'pos': '名詞,一般,*,*', 'base': '列'},
 {'surface': 'を', 'pos': '助詞,格助詞,一般,*', 'base': 'を'},
 {'surface': '形態素', 'pos': '名詞,一般,*,*', 'base': '形態素'},
 {'surface': '解析', 'pos': '名詞,サ変接続,*,*', 'base': '解析'},
 {'surface': 'し', 'pos': '動詞,自立,*,*', 'base': 'する'},
 {'surface': 'ます', 'pos': '助動詞,*,*,*', 'base': 'ます'},
 {'surface': '!!', 'pos': '記号,一般,*,*', 'base': '!!'}]

### 不要な形態素の除外

重要な意味を持たなかったりノイズになったりする形態素（ストップワード）を除外します。  
今回用いた除外方法は次の2つです。  

1. 重要でない品詞は除外する。
1. 多くの文章に現れている形態素を除外する。

全説明文に現れる形態素と品詞を出現頻度の高い順にみて決めました。  
今回、結果的に重要ではないとした品詞は以下 stop_pos のもの、重要でない形態素は「イラスト」「する」「(」「)」の4つと「!」や「?」で構成される形態素にしました。

今回はこのような人力で採用・不採用を決めてゼロイチでお重み付けする素朴な方法を用いましたが、他にもSCDVのように出現頻度情報（古典的にはTF-IDF）で重み付けしたり、固有表現やアテンションの類で形態素の重要度を重み付けする方法を使えばもっと精度が上がるかもしれません。もちろん、試してみないと実感に合うかどうか分かりませんし、トレードオフ（用途の向き不向き、非機能面の改悪）もあるでしょう。

In [15]:
stop_pos = {
    "助詞,格助詞,一般,*",
    "助詞,格助詞,引用,*",
    "助詞,格助詞,連語,*",
    "助詞,係助詞,*,*",
    "助詞,終助詞,*,*",
    "助詞,接続助詞,*,*",
    "助詞,特殊,*,*",
    "助詞,副詞化,*,*",
    "助詞,副助詞,*,*",
    "助詞,副助詞／並立助詞／終助詞,*,*",
    "助詞,並立助詞,*,*",
    "助詞,連体化,*,*",
    "助動詞,*,*,*",
    "記号,句点,*,*",
    "記号,読点,*,*",
    "記号,空白,*,*",
    "記号,一般,*,*",
    "記号,アルファベット,*,*",
    "記号,一般,*,*",
    "記号,括弧開,*,*",
    "記号,括弧閉,*,*",
    "動詞,接尾,*,*",
    "動詞,非自立,*,*",
    "名詞,非自立,一般,*",
    "名詞,非自立,形容動詞語幹,*",
    "名詞,非自立,助動詞語幹,*",
    "名詞,非自立,副詞可能,*",
    "名詞,接尾,助動詞語幹,*",
    "名詞,接尾,人名,*",
    "接頭詞,名詞接続,*,*"
}

vocab = {}
for item in items:
    desc = item["desc"]
    title = item["title"]
    tokens = tokenize(desc)
    for token in tokens:
        key = token.base
        pos = token.pos
        is_stop = pos in stop_pos
        v = vocab.get(key, { "count": 0, "pos": pos , "stop": is_stop})
        v["count"] += 1
        vocab[key] = v

vocab_list = []
for k in vocab:
    v = vocab[k]
    if not v["stop"]:
        vocab_list.append((v["count"], k, v["pos"], v["stop"]))

stop_posに含まれない品詞の形態素を出現頻度の高い順に一覧化します。  
タプルの情報は、左から出現頻度、形態素の原形、品詞、stop_posに含まれるか否か（Falseのみ）です。

In [16]:
vocab_list = sorted(vocab_list, reverse=True)
vocab_list[:10]

[(24276, 'イラスト', '名詞,一般,*,*', False),
 (10287, 'する', '動詞,自立,*,*', False),
 (2927, '女性', '名詞,一般,*,*', False),
 (2653, '男性', '名詞,一般,*,*', False),
 (2148, '使う', '動詞,自立,*,*', False),
 (1411, '人', '名詞,接尾,一般,*', False),
 (1401, '着る', '動詞,自立,*,*', False),
 (1346, 'なる', '動詞,自立,*,*', False),
 (1090, '描く', '動詞,自立,*,*', False),
 (1073, '中', '名詞,接尾,副詞可能,*', False)]

出現頻度が多い、トップ2の形態素は意味を持たないと考えて除外します。

In [17]:
stop_word = [w[1] for w in vocab_list[:2]]
stop_word

['イラスト', 'する']

mecabでは記号が"名詞,サ変接続,*,*"になることがあるため、!や?で構成される形態素も除外します。もっとパターンを増やした方がいいでしょう。

In [18]:
import re
stop_word_regex = [ re.compile("^[!?]+$")]

### 文の分散表現の計算方法の定義

与えられた文を、文の分散表現に変換する関数 get_sentence_vector を定義します。  
今回採用した文の分散表現の計算方法は次の通りです。

1. 正規化・形態素解析・前処理を行う。
2. 1で求めた形態素列の各形態素に対応する分散表現をfastTextを用いて計算する。
3. 2で求めた単語の分散表現の単純和を文の分散表現とする。

今回は形態素をそのまま入力にして学習したfastTextを使用するため、形態素そのままを入力にして分散表現を計算しています。もし、word2vec等で形態素の原形を用いて学習した場合は、形態素の原形を入力に分散表現を計算するといいでしょう。  
今回は（意図して）極めて素朴な文の分散表現計算方法（単語の分散表現の単純和）を採用しましたが、他にも（あまり精度は変わらないかもしれませんが）doc2vecや、より高度なニューラル言語モデル（[sentence-BERT](https://qiita.com/sonoisa/items/1df94d0a98cd4f209051)）などを用いて深い文脈情報を持った文の分散表現を作ってもいいでしょう。

In [19]:
import numpy as np

def get_sentence_vector(sentence):
    tokens = tokenize(sentence)
    vecs = []
    for token in tokens:
        if is_stop(token):
            continue
        surface = token.surface
        v = fasttext_model.query(surface)
#         v = v / np.linalg.norm(v, axis=0, ord=2)
        vecs.append(v)

    sent_vec = None
    for vec in vecs:
        if sent_vec is None:
            sent_vec = vec
        else:
            sent_vec = sent_vec + vec
    return sent_vec

def is_stop(token):
    return token.pos in stop_pos or token.base in stop_word or any([r for r in stop_word_regex if r.match(token.base) is not None])

試しに文ベクトルを計算してみます。

In [20]:
get_sentence_vector("与えられた文から文の分散表現を計算します。")

array([-1.5151137e-01, -3.1349602e-01,  6.7971635e-01,  1.0311966e+00,
        2.8439978e-01,  6.2196982e-01,  9.4056803e-01,  1.9055775e+00,
        9.5409608e-01, -8.8091004e-01,  1.7558415e-01, -1.5682095e+00,
       -6.8752211e-01,  9.9586457e-02,  1.0331454e+00,  9.7016275e-01,
       -3.9063269e-01, -2.7099931e-01,  8.2853079e-02, -5.2743930e-01,
        8.0596018e-01,  7.0612162e-02, -3.1877765e-01,  3.8583285e-01,
       -9.5091981e-01, -1.3337028e+00, -1.8184139e+00,  5.0215793e-01,
       -3.6932892e-01,  1.9496541e-01, -5.4951578e-01, -7.2261202e-01,
        3.1510192e-01,  7.1649033e-01, -1.6657740e+00, -1.3129126e+00,
        1.2015147e+00,  1.4813354e+00,  4.0738815e-01,  6.4084929e-01,
        8.1722784e-01,  8.0454010e-01,  2.0863664e-01, -1.3490838e-01,
       -8.1304508e-01, -1.0982885e+00, -1.1494877e-01,  1.0799465e+00,
        3.8322967e-01, -8.1491220e-01, -7.1751511e-01, -1.0450217e+00,
        3.8744628e-01,  1.0822784e+00,  8.9399111e-01,  2.8967397e+00,
      

## 説明文の分散表現の計算実行

画像メタデータに説明文の分散表現を追加します。

In [21]:
from tqdm import tqdm
for item in tqdm(items):
    desc = item["desc"]
    desc_vec = get_sentence_vector(desc)
    item["vec"] = desc_vec

100%|██████████| 25007/25007 [00:35<00:00, 698.75it/s]


## コサイン類似度の定義

今回は、文の意味の近さを、文の分散表現のコサイン類似度によって測ります。  
文の意味が近ければ、文の分散表現（ベクトル）v1とv2が近くなるという定性的性質を、ベクトルの成す角のcosによって測るということです。

In [22]:
import numpy as np

def cos_sim(v1, v2):
    v1 = v1 / np.linalg.norm(v1, axis=0, ord=2)
    v2 = v2 / np.linalg.norm(v2, axis=0, ord=2)
    return np.sum(v1 * v2)

## 画像検索結果GUIの定義

最後のステップです。画像を検索する関数を定義します。  
いままで作った関数を使えば、次の処理からなる検索アルゴリズム（最初の図も参照）を簡単に実装できますね。  

1. 与えられた文から文の分散表現を計算する。
2. その分散表現と、説明文の分散表現の間のコサイン類似度を計算する。
3. コサイン類似度の高い順に画像の関連情報を表示する。

**※なお「いらすとや」さんの広告収入モデルに悪影響を与えないよう、必ず「いらすとや」さんのページへのリンクを張り、画像のダウンロードは「いらすとや」さんのページから行うようにしましょう。その他、[「いらすとや」さんの利用規約](https://www.irasutoya.com/p/terms.html)に違反しないよう十分ご注意ください。**


In [23]:
from IPython.display import display, HTML, clear_output
from html import escape
import numpy as np

def search_irasuto(sentence, top_n=3):
    sentence_vector = get_sentence_vector(sentence)
    sims = []
    if sentence_vector is None:
        print("検索できない文章です。もう少し文章を長くしてみてください。")
    else:
        for item in items:
            v = item["vec"]
            if v is None:
                sims.append(-1.0)
            else:
                sim = cos_sim(sentence_vector, v)
                sims.append(sim)
    
    count = 0
    for index in np.argsort(sims)[::-1]:
        if count >= top_n:
            break
        item = items[index]
        desc = escape(item["desc"])
        imgs = item["imgs"]
        if len(imgs) == 0:
            continue
        img = imgs[0]
        page = item["page"]
        sim = sims[index]
        display(HTML("<div><a href='" + page + "' target='_blank' rel='noopener noreferrer'><img src='" + img + "' width='100'>" + str(sim) + ": " + desc + "</a><div>"))
        count += 1

## アプリの動作確認

さあ、これでアルゴリズムは完成しました。早速、試してみましょう。  

※今回の94件の解説用データは「AI, つづく, 拍手, 祈る, 笑い, お礼」という単語を説明文に含む画像に限定したもののため、これらの単語に関する検索のみが行えます。それでも十分アルゴリズムについて理解できるでしょう。

In [24]:
search_irasuto(sentence="暴走したAI", top_n=5)

In [25]:
search_irasuto(sentence="リモートワークで勉強会", top_n=5)

In [26]:
search_irasuto(sentence="いらすとやさんに惜しみない拍手を", top_n=1)